## Orchestration

Rapidement une application utilisant des conteneurs demande d'en coordonner plusieurs c'est que l'on appelle l'orchestration.
L'outil classique est [kubernetes](https://kubernetes.io/) de Google. Il s'agit d'un outils complexe. Une alternative simple est [docker compose](https://docs.docker.com/compose/).  

```{literalinclude} sample-java/restjpa/Docker-compose.yml
:language: docker-compose
:caption: sample-java/restjpa/Docker-compose.yml
:name: sample-java/restjpa/Docker-compose.yml
```

In [25]:
( cd sample-java/restjpa && docker compose down --volumes --remove-orphans && echo done )

[+] Running 0/0
 ⠋ Container restjpa-app-1  Stopping                                       0.0s
[+] Running 0/1
 ⠙ Container restjpa-app-1  Stopping                                       0.1s
[+] Running 0/1
 ⠹ Container restjpa-app-1  Stopping                                       0.2s
[+] Running 0/1
 ⠸ Container restjpa-app-1  Stopping                                       0.3s
[+] Running 0/1
 ⠼ Container restjpa-app-1  Stopping                                       0.4s
[+] Running 0/1
 ⠴ Container restjpa-app-1  Stopping                                       0.5s
[+] Running 0/1
 ⠦ Container restjpa-app-1  Stopping                                       0.6s
[+] Running 0/1
 ⠧ Container restjpa-app-1  Stopping                                       0.7s
[+] Running 1/1
 ⠿ Container restjpa-app-1  Removed                                        0.8s
 ⠋ Container restjpa-db-1   Stopping                                       0.0s
[+] Running 1/2
 ⠿ Container restjpa-app-1  Removed     

In [26]:
docker network create restjpa_frontend

Error response from daemon: network with name restjpa_frontend already exists


In [27]:
( cd sample-java/restjpa && \
  docker compose build --progress=quiet && \
  docker compose up --quiet-pull --detach && echo "done" )

[+] Running 2/0
 ⠿ Network restjpa_backend           Created                               0.0s
 ⠿ Volume "restjpa_restjpa-pg-data"  Created                               0.0s
 ⠋ Container restjpa-db-1            Creating                              0.0s
[+] Running 3/3
 ⠿ Network restjpa_backend           Created                               0.0s
 ⠿ Volume "restjpa_restjpa-pg-data"  Created                               0.0s
 ⠿ Container restjpa-db-1            Starting                              0.1s
 ⠿ Container restjpa-app-1           Created                               0.0s
[+] Running 3/4
 ⠿ Network restjpa_backend           Created                               0.0s
 ⠿ Volume "restjpa_restjpa-pg-data"  Created                               0.0s
 ⠿ Container restjpa-db-1            Starting                              0.2s
 ⠿ Container restjpa-app-1           Created                               0.0s
[+] Running 3/4
 ⠿ Network restjpa_backend           Created            

In [28]:
( cd sample-java/restjpa && docker compose logs )

restjpa-db-1  | The files belonging to this database system will be owned by user "postgres".
restjpa-db-1  | This user must also own the server process.
restjpa-db-1  | 
restjpa-db-1  | The database cluster will be initialized with locale "en_US.utf8".
restjpa-db-1  | The default database encoding has accordingly been set to "UTF8".
restjpa-db-1  | The default text search configuration will be set to "english".
restjpa-db-1  | 
restjpa-db-1  | Data page checksums are disabled.
restjpa-db-1  | 
restjpa-db-1  | fixing permissions on existing directory /var/lib/postgresql/data ... ok
restjpa-db-1  | creating subdirectories ... ok
restjpa-db-1  | selecting dynamic shared memory implementation ... posix
restjpa-db-1  | selecting default max_connections ... 100
restjpa-db-1  | selecting default shared_buffers ... 128MB
restjpa-db-1  | selecting default time zone ... UTC
restjpa-db-1  | creating configuration files ... ok
restjpa-db-1  | running bootstrap script ... ok
restjpa-db-1  | sh: lo

In [ ]:
docker run --network restjpa_frontend \
           --rm curlimages/curl:7.88.1 \
               curl --verbose --silent \
                 -H "Accept: application/json" \
                 http://restjpa-app-1:8080/restjpa/persons

*   Trying 172.23.0.2:8080...
* Connected to restjpa-app-1 (172.23.0.2) port 8080 (#0)
> GET /restjpa/persons HTTP/1.1
> Host: restjpa-app-1:8080
> User-Agent: curl/7.88.1-DEV
> Accept: application/json
> 
< HTTP/1.1 200 OK
< Content-Type: application/json
< Content-Length: 187
< 
{ [187 bytes data]
* Connection #0 to host restjpa-app-1 left intact
